Load Sloth

In [1]:
import slothpy as slt

Exception reporting mode: Plain


Import additional packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt

Create Compound from Orca output file

In [ ]:
DyCo = slt.compound_from_orca(".", "DyCo_demo", "QZVPP", ".", "DyCo_cif_cas_nevpt2_QZVPP.out")

You can add different calculations

In [ ]:
DyCo = slt.compound_from_orca(".", "DyCo_demo", "QZVPP_NEVPT2", ".", "DyCo_cif_cas_nevpt2_QZVPP.out", pt2 = True)

or add computations from different softwares etc.

In [ ]:
DyCo = slt.compound_from_molcas(".", "DyCo_demo", "TVZP_MOLCAS", ".", "DyCo_benchmark_aniso")

The last option is to load from the existing .slt file to resume the work

In [3]:
DyCo = slt.compound_from_slt(".", "DyCo_demo")

You can see what is inside the current file

In [4]:
DyCo

Compound from File "./HoCo.slt" with the following Groups of data:
bas3: {'Description': 'Group(bas3) containing results of relativistic SOC MOLCAS calculations - angular momenta, spin in SOC basis and SOC energies'}
bas31: {}
bas311: {}
bas3_3d_hemholtz_energy: {'Description': 'Group(bas3) containing 3D hemholtz_energy calculated from group: bas3.'}
bas3_3d_magnetisation: {'Description': 'Group(bas3) containing 3D magnetisation calculated from group: bas3.'}
bas3_3d_susceptibility: {'Description': 'Group(bas3) containing 3D magnetic susceptibility calculated from group: bas3.'}
dupaa: {'Description': 'lol'}
dupaaa: {'Description': 'lol'}
dupaaaaaaaa: {'Description': 'lols'}
new Api_g_tensors_axes: {'Description': 'Group(new Api) containing g-tensors of doublets and their magnetic axes calculated from Group: bas3.'}
and Datasets:
bas3/SOC_LX
bas3/SOC_LY
bas3/SOC_LZ
bas3/SOC_SX
bas3/SOC_SY
bas3/SOC_SZ
bas3/SOC_energies
bas31/trokorka
bas311/trokorka
bas3_3d_hemholtz_energy/bas3_energy_3

Now we can calculate the first properties. Lets start with molar magnetisation from a chosen group of data

In [ ]:
fields_mag = np.linspace(0.0001, 7, 64)
temperatures_mag = np.linspace(1, 5, 5)

In [ ]:
%%time
magnetisation = DyCo.calculate_mth("QZVPP", 252, fields_mag, 5, temperatures_mag, 64, 2, slt = "QZVPP")

In [ ]:
magnetisation

You can delete a chosen group if you know its suffix in order to be able to use the same name again

In [ ]:
DyCo

In [ ]:
DyCo.delete_group("QZVPP_magnetisation")

In [ ]:
DyCo

You can access data sets directly using custom interface similar to that for arrays

In [ ]:
mth = DyCo["QZVPP_magnetisation", "QZVPP_mth"]
fields = DyCo["QZVPP_magnetisation", "QZVPP_fields"]
temperatures = DyCo["QZVPP_magnetisation", "QZVPP_temperatures"]  

Now we can work using the data. Lets plot it!

In [ ]:
for mt in mth:
    plt.plot(fields, mt)

Or just use the prepared methods for plotting

In [ ]:
DyCo.plot_mth("QZVPP")

We can now do the same for susceptibility

In [ ]:
fields_chi = np.linspace(0.1, 3.1, 3)
temperatures_chi = np.linspace(1, 300, 300)

In [ ]:
%%time
chitht = DyCo.calculate_chitht("QZVPP", 1024, temperatures_chi, fields_chi, 3, 0.0001, 64, 2, slt = "QZVPP")

In [ ]:
for chith in chitht:
    plt.plot(temperatures_chi, chith)

In [ ]:
DyCo.plot_chitht("QZVPP")

To obtain energies of spin-orbit states you can run:

In [ ]:
DyCo.soc_energies_cm_1("QZVPP", 16)

You can look at the Zeeman splitting for various directions of the applied magnetic field or powder-average

In [ ]:
fields_zeem = np.linspace(0, 7, 64)

In [ ]:
zeeman = DyCo.calculate_zeeman_splitting("QZVPP", 256, 8, fields_zeem, [[0.,0.,1.], [0., 1., 0.], [1.,0.,1.]], 64, 2, slt = "QZVPP")

In [ ]:
DyCo.plot_zeeman("QZVPP")

There is an option to calculate pseudo-g-tensors for doublet states together with rotation matrices to the framework of main magnetic axes that can be used in many functions to rotate magnetic and angular momenta.

In [ ]:
g_tensor, axes = DyCo.calculate_g_tensor_and_axes_doublet("QZVPP_NEVPT2", [0,1,2,3], slt = "QZVPPP")

In [ ]:
g_tensor

In [ ]:
axes

New features include various properties (magnetisation, susceptibility, Hemholtz energy) calculated for their angular dependence and visualised in 3D plots

In [ ]:
%%time
mag_3d = DyCo.calculate_mag_3d("QZVPP", 256, [1.], 100, [2.], 64, 2, slt = "QZVPP")

In [ ]:
%matplotlib widget
DyCo.plot_mag_3d("QZVPP", colour_map_name="dark_rainbow", r_density = 45, c_density = 45,  ticks=2)

In [ ]:
%%time
DyCo.animate_energy_3d("QZVPP", 64, [0.1], 100, 1, 30, 180, 64, 2, ticks = 10, r_density = 90, c_density = 90, fps = 20, filename = "DyCo_energy_3d")

In [ ]:
%%time
DyCo.animate_mag_3d("QZVPP", 64, [0.1], 100, 1, 30, 180, 64, 2, ticks = 10, r_density = 90, c_density = 90, fps = 20, filename = "DyCo_mag_3d")

We can get a decomposition of SO-states in pseudo-spin basis of total angular or magnetic momenta with the frame rotation to the main magnetic axis of the ground doublet

In [ ]:
DyCo.decomposition_in_z_total_angular_momentum_basis("TVZP_MOLCAS", 0, 15, rotation = np.array([[ 9.99993456e-01, -3.20017155e-03, -1.68724195e-03],
        [ 3.20053868e-03,  9.99994855e-01,  2.14937173e-04],
        [ 1.68654543e-03, -2.20335849e-04,  9.99998554e-01]]), slt = "QZVPPP")

In [ ]:
DyCo.decomposition_in_z_magnetic_momentum_basis("QZVPP", 0, 15, rotation = np.array([[ 9.99993456e-01, -3.20017155e-03, -1.68724195e-03],
        [ 3.20053868e-03,  9.99994855e-01,  2.14937173e-04],
        [ 1.68654543e-03, -2.20335849e-04,  9.99998554e-01]]), slt = "QZVPP")

Lets get now SOC and Zeeman matrices in pseudo-spin basis

In [ ]:
soc_matrix = DyCo.soc_zeem_in_angular_magnetic_momentum_basis("QZVPP", 0, 15, "soc", "angular", rotation = np.array([[ 9.99993456e-01, -3.20017155e-03, -1.68724195e-03],
        [ 3.20053868e-03,  9.99994855e-01,  2.14937173e-04],
        [ 1.68654543e-03, -2.20335849e-04,  9.99998554e-01]]))

In [ ]:
zeeman_matrix = DyCo.soc_zeem_in_angular_magnetic_momentum_basis("QZVPP", 0, 15, "zeeman", "angular", field=1, orientation=np.array([0.,0.,1.]), rotation = np.array([[ 9.99993456e-01, -3.20017155e-03, -1.68724195e-03],
        [ 3.20053868e-03,  9.99994855e-01,  2.14937173e-04],
        [ 1.68654543e-03, -2.20335849e-04,  9.99998554e-01]]))

and ITO-decompose them (show both even and odd order)

In [ ]:
cfp = DyCo.soc_crystal_field_parameters("QZVPP", 0, 15, 14, slt = "QZVPPPPPP")

cfp_cm_1 = np.array(cfp)

for i in range(cfp_cm_1 .shape[0]):
    cfp_cm_1 [i,2] = cfp_cm_1 [i,2] * 219474.6
    
cfp_cm_1

In [ ]:
cfp = DyCo.soc_crystal_field_parameters("QZVPP", 0, 15, 14, slt = "QZVPPPPPPP", rotation = np.array([[ 0., 1., 0.],
        [ 1.,  0.,  0.],
        [ 0., 0.,  1.]]))

cfp_cm_1 = np.array(cfp)

for i in range(cfp_cm_1 .shape[0]):
    cfp_cm_1 [i,2] = cfp_cm_1 [i,2] * 219474.6
    
cfp_cm_1

In [ ]:
ito_soc_matrix = DyCo.matrix_from_ito("QZVPPP")

In [ ]:
DyCo.soc_energies_cm_1("QZVPP", 16)

In [ ]:
energies, eigenvectors = np.linalg.eigh(ito_soc_matrix)
energies * 219474.6

Note: without fixing the phase convention of pseudo-spin eigenstates even order was failing!

In [11]:
ito = DyCo.zeeman_matrix_ito_decpomosition("QZVPP", 0, 15, 1, np.array([0.,1.,0.]), 15)

In [12]:
ito_cm_1 = np.array(ito)

for i in range(ito_cm_1 .shape[0]):
    ito_cm_1 [i,2] = ito_cm_1 [i,2] * 219474.6
    
ito_cm_1

array([[ 0.00000000e+00,  0.00000000e+00,  2.57855615e+02],
       [ 1.00000000e+00, -1.00000000e+00,  8.75411658e-01],
       [ 1.00000000e+00,  0.00000000e+00, -5.22707456e-07],
       [ 1.00000000e+00,  1.00000000e+00, -2.76993390e-06],
       [ 2.00000000e+00, -2.00000000e+00, -6.86808399e-04],
       [ 2.00000000e+00, -1.00000000e+00, -1.74166084e-03],
       [ 2.00000000e+00,  0.00000000e+00, -8.19783796e+00],
       [ 2.00000000e+00,  1.00000000e+00,  2.86822953e-03],
       [ 2.00000000e+00,  2.00000000e+00, -8.32356305e-03],
       [ 3.00000000e+00, -3.00000000e+00, -1.07288619e-05],
       [ 3.00000000e+00, -2.00000000e+00,  6.17710702e-08],
       [ 3.00000000e+00, -1.00000000e+00,  4.57246508e-05],
       [ 3.00000000e+00,  0.00000000e+00,  2.57471927e-08],
       [ 3.00000000e+00,  1.00000000e+00, -1.02017931e-08],
       [ 3.00000000e+00,  2.00000000e+00,  3.01031037e-08],
       [ 3.00000000e+00,  3.00000000e+00,  1.48089310e-08],
       [ 4.00000000e+00, -4.00000000e+00

In [ ]:
ito_zeeman_matrix = DyCo.matrix_from_ito("QZVPP_zeem")

In [ ]:
energies, eigenvectors = np.linalg.eigh(zeeman_matrix)
energies * 219474.6

In [ ]:
energies, eigenvectors = np.linalg.eigh(ito_zeeman_matrix)
energies * 219474.6

In [ ]:
ito_zeeman_matrix - zeeman_matrix

Show it without rotation (here it is almost negligible)

Lets jump to the Spiral and check my parameters

In [ ]:
index = 0

for k in range(0, 16, 2):
    print(f"&END")
    print(f"&DEF_O 1 {k}")
    for q in range(-k, k+1):
        print(f"{q} {cfp_cm_1[index][2]}")
        index += 1